In [5]:
from lightgbm import LGBMRegressor
import pandas as pd
from typing import Dict, Tuple
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import logging
from sklearn.model_selection import GridSearchCV

logger = logging.getLogger(__name__)

In [6]:
catalog.list()

[
    'companies',
    'reviews',
    'shuttles',
    'preprocessed_companies',
    'preprocessed_shuttles',
    'model_input_table',
    'regressor',
    'X_train',
    'y_train',
    'X_test',
    'y_test',
    'parameters',
    'params:model_options',
    'params:model_options.test_size',
    'params:model_options.random_state',
    'params:model_options.target',
    'params:model_options.max_depth',
    'params:model_options.features'
]

In [7]:
df=catalog.load('model_input_table')

[03/09/23 10:21:13] INFO     Loading data from 'model_input_table' (ParquetDataSet)...          ]8;id=792202;file:///opt/anaconda3/lib/python3.9/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=195950;file:///opt/anaconda3/lib/python3.9/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [8]:
df.head(10)

,id_x,shuttle_location,shuttle_type,engine_type,engine_vendor,engines,passenger_capacity,cancellation_policy,crew,d_check_complete,...,review_scores_crew,review_scores_location,review_scores_price,number_of_reviews,reviews_per_month,id_y,company_rating,company_location,total_fleet_count,iata_approved
0,63561,Niue,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,False,...,10.0,9.0,10.0,133,1.65,35029,1.0,Niue,4.0,False
1,63561,Niue,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,False,...,10.0,9.0,10.0,133,1.65,35029,1.0,Niue,4.0,False
2,63561,Niue,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,False,...,10.0,9.0,10.0,133,1.65,35029,1.0,Niue,4.0,False
3,63561,Niue,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,False,...,10.0,9.0,10.0,133,1.65,35029,1.0,Niue,4.0,False
4,53260,Niue,Type V5,Quantum,"Banks, Wood and Phillips",1.0,2,strict,1.0,False,...,10.0,9.0,10.0,37,0.48,35029,1.0,Niue,4.0,False
5,53260,Niue,Type V5,Quantum,"Banks, Wood and Phillips",1.0,2,strict,1.0,False,...,10.0,9.0,10.0,37,0.48,35029,1.0,Niue,4.0,False
6,53260,Niue,Type V5,Quantum,"Banks, Wood and Phillips",1.0,2,strict,1.0,False,...,10.0,9.0,10.0,37,0.48,35029,1.0,Niue,4.0,False
7,53260,Niue,Type V5,Quantum,"Banks, Wood and Phillips",1.0,2,strict,1.0,False,...,10.0,9.0,10.0,37,0.48,35029,1.0,Niue,4.0,False
8,51019,Niue,Type V5,Quantum,ThetaBase Services,1.0,2,flexible,1.0,False,...,10.0,9.0,9.0,10,0.15,35029,1.0,Niue,4.0,False
9,51019,Niue,Type V5,Quantum,ThetaBase Services,1.0,2,flexible,1.0,False,...,10.0,9.0,9.0,10,0.15,35029,1.0,Niue,4.0,False


In [9]:
X_train=catalog.load('X_train')

[03/09/23 10:26:15] INFO     Loading data from 'X_train' (PickleDataSet)...                     ]8;id=434476;file:///opt/anaconda3/lib/python3.9/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=451274;file:///opt/anaconda3/lib/python3.9/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [10]:
y_train=catalog.load('y_train')

[03/09/23 10:26:28] INFO     Loading data from 'y_train' (PickleDataSet)...                     ]8;id=57271;file:///opt/anaconda3/lib/python3.9/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=986270;file:///opt/anaconda3/lib/python3.9/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [12]:
y_train.head(10)

150544     2950.0
508525     5290.0
914264     3418.0
1533279    2885.0
1097861    2846.0
141114     2430.0
154690     1975.0
608456     5186.0
502225     5004.0
649379     4458.0
Name: price, dtype: float64

In [13]:
def train_linear_regression_model(X_train: pd.DataFrame, y_train: pd.Series) -> LinearRegression:
    regressor = LinearRegression()
    regressor.fit(X_train, y_train)
    return regressor

In [15]:
lr=train_linear_regression_model(X_train, y_train)

In [16]:
lr

LinearRegression()

In [17]:
def evaluate_model(regressor: LinearRegression):
    X_test=catalog.load('X_test')
    y_test=catalog.load('y_test')
    y_pred = regressor.predict(X_test)
    score = r2_score(y_test, y_pred)  #Correlacion
    print('Model has a coefficient R^2 of %.3f on test data.' %score)
    

In [18]:
evaluate_model(lr)

[03/09/23 10:33:43] INFO     Loading data from 'X_test' (PickleDataSet)...                      ]8;id=416490;file:///opt/anaconda3/lib/python3.9/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=98696;file:///opt/anaconda3/lib/python3.9/site-packages/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Loading data from 'y_test' (PickleDataSet)...                      ]8;id=778571;file:///opt/anaconda3/lib/python3.9/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=456743;file:///opt/anaconda3/lib/python3.9/site-packages/kedro/io/data_catalog.py#343\343]8;;\

Model has a coefficient R^2 of 0.462 on test data.


In [19]:
def train_random_forest_regression_model(X_train: pd.DataFrame, y_train: pd.Series) -> RandomForestRegressor:
    var_max_depth= catalog.load('params:model_options.max_depth')
    rf_regressor = RandomForestRegressor(max_depth=var_max_depth)
    rf_regressor.fit(X_train, y_train)
    return rf_regressor

In [20]:
fr=train_random_forest_regression_model(X_train, y_train)

[03/09/23 10:35:09] INFO     Loading data from 'params:model_options.max_depth'                 ]8;id=128152;file:///opt/anaconda3/lib/python3.9/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=926277;file:///opt/anaconda3/lib/python3.9/site-packages/kedro/io/data_catalog.py#343\343]8;;\
                             (MemoryDataSet)...                                                                    

In [21]:
fr

RandomForestRegressor(max_depth=25)

In [22]:
evaluate_model(fr)

[03/09/23 10:37:26] INFO     Loading data from 'X_test' (PickleDataSet)...                      ]8;id=365567;file:///opt/anaconda3/lib/python3.9/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=914147;file:///opt/anaconda3/lib/python3.9/site-packages/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Loading data from 'y_test' (PickleDataSet)...                      ]8;id=96383;file:///opt/anaconda3/lib/python3.9/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=43441;file:///opt/anaconda3/lib/python3.9/site-packages/kedro/io/data_catalog.py#343\343]8;;\

Model has a coefficient R^2 of 0.797 on test data.


In [35]:
params={'max_depth':5, 'n_estimators': 1000, 'learning_rate': 0.05}

In [36]:
def train_lgbm_regression_model(X_train: pd.DataFrame, y_train: pd.Series, params:Dict) -> LGBMRegressor:
    
    lgbm_regressor = LGBMRegressor(**params)
    lgbm_regressor.fit(X_train, y_train)
    return lgbm_regressor

In [40]:
lgbm=train_lgbm_regression_model(X_train, y_train,params)

In [41]:
lgbm

LGBMRegressor(learning_rate=0.05, max_depth=5, n_estimators=1000)

In [42]:
evaluate_model(lgbm)

[03/09/23 10:52:40] INFO     Loading data from 'X_test' (PickleDataSet)...                      ]8;id=528246;file:///opt/anaconda3/lib/python3.9/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=554278;file:///opt/anaconda3/lib/python3.9/site-packages/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Loading data from 'y_test' (PickleDataSet)...                      ]8;id=739950;file:///opt/anaconda3/lib/python3.9/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=420699;file:///opt/anaconda3/lib/python3.9/site-packages/kedro/io/data_catalog.py#343\343]8;;\

Model has a coefficient R^2 of 0.744 on test data.


In [28]:
params_grid = { 
    'max_depth': [15, 20, 25,30]
}

In [29]:
reg_test=GridSearchCV (RandomForestRegressor(), param_grid=params_grid )

In [30]:
reg_test.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'max_depth': [15, 20, 25, 30]})

In [38]:
reg_test.best_params_

{'max_depth': 30}

In [39]:
reg_test.best_score_

0.7938580231670549

Model has a coefficient R^2 of 0.717 on test data.
